In [1]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from pandas import read_csv
import numpy as np
import operator
from getData import getData
from getConfig import getConfig
from getETSparms import getETSparms
from calcMAPE import calcMAPE

In [2]:
# Input is the time series. Split into Train and Test
def train_test_split(ts, config):
    steps = config["testSize"]
    start = config["trainingSize"]+steps
    train = ts[-start:-steps]
    test = ts[-steps:]
    train = np.array(train)
    test = np.array(test)
    return train, test

In [3]:
def buildModel(train, m):
    '''
    Build a Holt-Winters ETS model with passed parameters
    '''
    t = m["trend"]
    d = m["damped"]
    s = m["seasonal"]
    p = m["periods"]
    return ExponentialSmoothing(\
                                train,\
                                trend=t,\
                                damped=d,\
                                seasonal=s,\
                                seasonal_periods=p)

In [4]:
def fitModel(model, f):
    '''
    Fit the model
    '''
    b = f["BoxCox"]
    r = f["RemoveBias"]
    alpha = f["smoothing_level"]
    return model.fit(\
                     optimized=True,\
                     use_boxcox=b,\
                     remove_bias=r,\
                     smoothing_level=alpha)

In [5]:
def predict(model, config):
    '''
    Using the input model, make predictions using "forecast"
    '''
    steps = config["testSize"]
    preds = model.forecast(steps=steps)
    return preds

In [6]:
def savePrediction(test, preds):
    # Store the results of the prediction
    test = np.reshape(test, [-1,1])
    
    preds = np.reshape(preds, [-1,1])
    
    merged = np.concatenate([test, preds], axis=1)
    fname = "predictions.csv"
    np.savetxt(config["dataLoc"]+fname, merged, delimiter=",", fmt='%9.2f', header="Test, Prediction")

In [7]:
config = getConfig()
df = getData(config)
ts = df[config["labelColumn"]]
train, test = train_test_split(ts, config)

In [8]:
modelParms, fitParms = getETSparms()
total = len(modelParms) * len(fitParms)
assert (total == 1), "This module only designed to work with a single set of parameters"

In [10]:
model = buildModel(train, modelParms[0])
model = fitModel(model, fitParms[0])
preds = predict(model, config)
error = calcMAPE(test, preds)
print(error)
savePrediction(test, preds)

0.011962817718576185


In [11]:
t = np.reshape(train, [-1,1])
f = np.reshape(model.fittedvalues, [-1,1])
x = np.concatenate((t, f), axis=1)

In [12]:
np.savetxt(config["dataLoc"]+"fitted.csv", x, delimiter=",", header="Train,Fitted")